In [ ]:
import cmath
import operator
import os
import sys
from types import SimpleNamespace

import adaptive
import matplotlib.pyplot as plt
import numpy as np

sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
adaptive.notebook_extension()

In [ ]:
def k_j(p, j):
    op = {1: operator.sub,
          2: operator.add}[j]
    p.E_z = 0.5 * p.mu_B * p.g * p.B
    sqrt_term = np.sqrt(
        p.E_z**2
        - 2 * p.alpha * p.E_z * np.sin(p.theta) * p.k_x + p.alpha**2 * p.k_x**2)
    k_sq = (2 * p.m_eff / p.hbar**2) * op(p.mu, sqrt_term) - p.k_x**2
    return np.sqrt(k_sq)

def a_min(z_x, z_y, W, *, correction=True):
    theta = np.arctan(4 * z_y / z_x)
    crossing_point = W / np.cos(theta)
    if z_y < crossing_point / 2:
        return np.nan
    a = z_x / 2
    b = z_y * 2
    c = np.sqrt(W**2 + (b/a)**2 * W**2)
    
    if correction:
        D = np.sqrt(a**2 + (b - c)**2)
    else:
        D = np.sqrt(a**2 + (b + c)**2)
    xprime = np.sqrt(D**2 - W**2)
    return W / xprime

def tan_a(params):
    p = SimpleNamespace(**params)
    ky1 = k_j(p, 1)
    ky2 = k_j(p, 2)
    Ez = p.g * p.mu_B * p.B
    term_1 = np.sqrt(Ez**2 + p.alpha * p.k_x * (p.alpha * p.k_x - 2 * Ez * np.sin(p.theta)))
    term_2 = p.alpha * p.m_eff * (p.alpha * p.k_x - Ez * np.sin(p.theta))
    
    ratio_1 = p.hbar**2 * ky1 / (term_2 / term_1 + p.hbar**2*p.k_x)
    ratio_2 = p.hbar**2 * ky2 / (-term_2 / term_1 + p.hbar**2*p.k_x)
    
    return ratio_1, ratio_2

def get_cutoff(z_x, z_y, W, params, **_):
    a_m = a_min(z_x, z_y, W)
    
    k_F = np.sqrt(params['mu'] * (2 * params['m_eff'])) / params['hbar']
    
    def f(k):
        return abs(np.min(tan_a(dict(params, k_x=k))) - a_m)

    k_c, e = scipy.optimize.fmin(f, k_F/2, ftol=1e-7, xtol=1e-6, full_output=1, disp=0)[:2]
    return k_c if e < 1e-3 else np.inf

In [ ]:
import sns_system, spectrum
import scipy.constants

constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
)


a = 1
W = 200
params = dict(g=26, alpha=20)
params = dict(params,
    g_factor_middle=params['g'],
    g_factor_left=0,
    g_factor_right=0,
    mu=40,
    alpha_middle=params['alpha'],
    alpha_left=0,
    alpha_right=0,
    Delta_left=1,
    Delta_right=1,
    B=0.4,
    B_x=None,
    B_y=None,
    B_z=0,
    phase=np.pi,
    theta=np.arctan(1/20),
    T=0.0,
    V=0,
    **constants
)

syst_pars = dict(
    L_m=200,
    L_x=a,
    L_sc_up=1600,
    L_sc_down=1600,
    z_x=a,
    z_y=0,
    a=a,
    shape=None,
    transverse_soi=True,
    mu_from_bottom_of_spin_orbit_bands=False,
    k_x_in_sc=True,
    wraparound=True,
    current=False,
    ns_junction=False
)

syst, _, _ = sns_system.make_system(**syst_pars)

In [ ]:
from functools import lru_cache, partial

def bands(kx, theta, params=params, syst=syst):
    params = dict(params,
                  theta=theta,
                  B_x=np.cos(theta)*params['B'],
                  B_y=np.sin(theta)*params['B'],
                  k_x=kx*a)
    return np.min(np.abs(spectrum.calc_spectrum(syst, params, k=2)[0]))

ratios = np.arange(4, 20, 2)
cdims = [{'ratio': r} for r in ratios]
k_F = np.sqrt(params['mu'] * (2 * params['m_eff'])) / params['hbar']
loss = adaptive.learner.learner1D.triangle_loss
learners = [adaptive.Learner1D(partial(bands, theta=np.arctan(4/ratio)),
                               [0, 1.1*k_F],
                               loss_per_interval=loss)
            for ratio in ratios]
bl = adaptive.BalancingLearner(learners)

In [ ]:
runner = adaptive.Runner(bl)
runner.live_info()

In [ ]:
bl.plot(cdims)

In [ ]:
def process_bandstructures(learner, ratios):
    gaps = {}
    
    for l, ratio in zip(learner.learners, ratios):
        data = l.data
        min_data = np.min(data.values())
        ks, energies = (np.array(list(data.keys())), np.array(list(data.values())))
        idx_sort = np.argsort(ks)
        ks = ks[idx_sort]
        energies = energies[idx_sort]
        
        z_y = np.linspace(100, 400, 100)
        W = syst_pars['L_m']
        k_max = [get_cutoff(ratio*zy, zy, W, params) for zy in z_y]

        temp_energies = np.interp(k_max, ks, np.minimum.accumulate(energies,0))
        gaps[ratio] = scipy.interpolate.interp1d(z_y, temp_energies)
        
    return gaps

gaps = process_bandstructures(bl, ratios)

In [ ]:
z_y = np.linspace(100, 400, 800)

fig, ax = plt.subplots(figsize=(10,10))

ax.plot(z_y, np.array([gaps[ratio](z_y) for ratio in ratios[::]]).T)
leg = [f'{np.arctan(4/ratio) / np.pi *180:1.0f}°' for ratio in ratios[::]]
plt.legend(leg);
plt.xlabel('z_y (nm)');
plt.ylabel('gap (E/Δ)');